In [1]:
import pandas as pd
import re
from datetime import datetime
import numpy as np
from grace_read.read import read
from grace_read.read_header import read_header
import os

root_dir = os.getcwd()+"../../Data/"
files_path = [os.getcwd()+"/../../Data/"+x for x in os.listdir("../../Data/")]

In [2]:
data = read(files_path)

/home/shivam/PycharmProjects/grace-fo-stda/tests/experimental/../../Data/kfilter_DDK1_GSM-2_2019305-2019334_GRFO_JPLEM_BA01_0600.gfc
/home/shivam/PycharmProjects/grace-fo-stda/tests/experimental/../../Data/kfilter_DDK1_GSM-2_2019152-2019181_GRFO_JPLEM_BA01_0600.gfc
/home/shivam/PycharmProjects/grace-fo-stda/tests/experimental/../../Data/kfilter_DDK1_GSM-2_2019244-2019273_GRFO_JPLEM_BA01_0600.gfc
/home/shivam/PycharmProjects/grace-fo-stda/tests/experimental/../../Data/kfilter_DDK1_GSM-2_2019274-2019304_GRFO_JPLEM_BA01_0600.gfc
/home/shivam/PycharmProjects/grace-fo-stda/tests/experimental/../../Data/kfilter_DDK1_GSM-2_2018335-2018365_GRFO_JPLEM_BA01_0600.gfc
/home/shivam/PycharmProjects/grace-fo-stda/tests/experimental/../../Data/kfilter_DDK1_GSM-2_2020122-2020152_GRFO_JPLEM_BA01_0600.gfc
/home/shivam/PycharmProjects/grace-fo-stda/tests/experimental/../../Data/kfilter_DDK1_GSM-2_2019335-2019365_GRFO_JPLEM_BA01_0600.gfc
/home/shivam/PycharmProjects/grace-fo-stda/tests/experimental/../../D

In [3]:
sc_coeffs = data["sc_coeffs_mat"][0]

In [5]:
np.nonzero(sc_coeffs)

(array([ 0,  2,  2, ..., 60, 60, 60]),
 array([ 0,  0,  0, ..., 60, 60, 60]),
 array([0, 0, 2, ..., 1, 2, 3]))

In [4]:
d[[0, 1],
[1, 1]]

array([0.79321211, 0.25413539])

In [5]:
np.take(d, [[0, 1],
[1, 1]])

array([[0.17441864, 0.79321211],
       [0.79321211, 0.79321211]])

In [6]:
d


array([[0.17441864, 0.79321211],
       [0.73959024, 0.25413539],
       [0.01126231, 0.96862342],
       [0.80224881, 0.24904885],
       [0.38697669, 0.63292706]])